In [1]:
import nibabel as nib
from scipy.io import savemat
from nilearn import *
import numpy as np
import os

In [3]:
# Path to folder with all masks
dir_path = r'.\masks\masks'

# Path to save all masks
save_path = './resampled_masks/1092_'

# list to store files
res = []
# Iterate directory
for file in os.listdir(dir_path):
    # check only text files
    if file.endswith('.nii'):
        res.append(file)

# One data image as reference
data_img = nib.load('../DATA/HU1092_27_11_13/MOVIE1_LONG/Nifti/wrsHU1092-0701-00007-000001-01.hdr')

In [4]:
# Iterate over all masks and resample them
for f in res:
    # Load data from img file
    mask_filename = './masks/masks/' + f
    mask_img = nib.load(mask_filename)

    # Adjust the affine of the mask to match the data
    mask_img_resampled = image.resample_to_img(mask_img, target_img=data_img, interpolation='nearest')

    print('Saving resampled version of '+f)
    
    # Save the mask
    nib.save(mask_img_resampled, save_path+f)

Saving resampled version of ACC_L.nii
Saving resampled version of ACC_R.nii
Saving resampled version of amygdala_l.nii
Saving resampled version of amygdala_r.nii
Saving resampled version of angular_l.nii
Saving resampled version of angular_r.nii
Saving resampled version of frontal_inf_oper_l.nii
Saving resampled version of frontal_inf_oper_r.nii
Saving resampled version of frontal_inf_orb_L.nii
Saving resampled version of frontal_inf_orb_r.nii
Saving resampled version of Frontal_Inf_Tri_l.nii
Saving resampled version of frontal_Inf_tri_r.nii
Saving resampled version of frontal_med_orb_l.nii
Saving resampled version of frontal_med_orb_r.nii
Saving resampled version of frontal_mid_l.nii
Saving resampled version of frontal_mid_orb_l.nii
Saving resampled version of frontal_mid_orb_r.nii
Saving resampled version of frontal_mid_r.nii
Saving resampled version of frontal_sup_L.nii
Saving resampled version of frontal_sup_medial_l.nii
Saving resampled version of frontal_sup_medial_r.nii
Saving r

In [ ]:
# Create a dictionary to store all the regions' matrices
data = {}

for f in res:
    # Load data from img file
    mask_filename = './masks/' + f
    mask_img = nib.load(mask_filename)

    # Adjust the affine of the mask to match the data
    mask_img = image.resample_to_img(mask_img, target_img=data_img, interpolation='nearest')
    
    mask_data = mask_img.get_fdata()
    
    # Get the affine matrix
    affine = mask_img.affine

    # Get indices where the mask is non-zero
    indices = np.argwhere(mask_data != 0)
    
    # Add homogeneous coordinate for the transformation
    indices_homogeneous = np.column_stack((indices, np.ones(indices.shape[0])))
    
    # Apply the affine transformation to convert to MNI space
    mni_coords = np.dot(affine, indices_homogeneous.T).T
    mni_coords = mni_coords[:, :3]  # Extract only x, y, z coordinates
    
    # Append the MNI coordinates for this mask to the dictionary
    data[f[:-4]] = mni_coords.astype(float)


In [4]:
# Save as MATLAB Data file
savemat("MNI_allMasks.mat", data)